<a href="https://colab.research.google.com/github/merchantrayyan/emotion_detection_sample/blob/main/sample_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sample model demonstrating the model works

Unfortunately installing all this stuff every time takes a few minutes but it's required

Weights are loaded from the files you upload

Three emotions are predicted with top 3 emotions and certainty for each inputted line

Task is hard, Google's F1 score is 46% even with all this data so I'm not sure how much we can improve upon this with hyperparameter tuning but maybe merging several emotions will help

We don't need all of the emotions anyways

In [ ]:
!pip install tensorflow==2.11.0

In [ ]:
!sudo apt install curl gnupg
!curl https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update
!sudo apt install bazel=5.4.0

In [ ]:
!git clone https://www.github.com/tensorflow/models
!models/research/seq_flow_lite/demo/colab/setup_workspace.sh
!pip install models/research/seq_flow_lite
!rm -rf models/research/seq_flow_lite/tf_ops
!rm -rf models/research/seq_flow_lite/tflite_ops

In [ ]:
%cd models/research/seq_flow_lite

In [ ]:
import tensorflow as tf

In [ ]:
from models import prado
from layers import base_layers
from layers import projection_layers

def build_model(mode):
  # First we define our inputs.
  inputs = []
  if mode == base_layers.TRAIN or mode == base_layers.EVAL:
    # For TRAIN and EVAL, we'll be getting dataset examples,
    # so we'll get projections and sequence_lengths.
    projection = tf.keras.Input(
        shape=(MODEL_CONFIG['max_seq_len'], MODEL_CONFIG['feature_size']),
        name='projection',
        dtype='float32')

    sequence_length = tf.keras.Input(
        shape=(), name='sequence_length', dtype='float32')
    inputs = [projection, sequence_length]
  else:
    # Otherwise, we get string inputs which we need to project.
    input = tf.keras.Input(shape=(), name='input', dtype='string')
    projection_layer = projection_layers.ProjectionLayer(MODEL_CONFIG, mode)
    projection, sequence_length = projection_layer(input)
    inputs = [input]

  # Next we add the model layer.
  model_layer = prado.Encoder(MODEL_CONFIG, mode)
  logits = model_layer(projection, sequence_length)

  # Finally we add an activation layer.
  if MODEL_CONFIG['multilabel']:
    activation = tf.keras.layers.Activation('sigmoid', name='predictions')
  else:
    activation = tf.keras.layers.Activation('softmax', name='predictions')
  predictions = activation(logits)

  model = tf.keras.Model(
      inputs=inputs,
      outputs=[predictions])

  return model

In [ ]:
LABELS = [
    'admiration',
    'amusement',
    'anger',
    'annoyance'
    'approval',
    'caring',
    'confusion',
    'curiosity',
    'desire',
    'disappointment',
    'disapproval',
    'disgust',
    'embarrassment',
    'excitement',
    'fear',
    'gratitude',
    'grief',
    'joy',
    'love',
    'nervousness',
    'optimism',
    'pride',
    'realization',
    'relief',
    'remorse',
    'sadness',
    'surprise',
    'neutral',
]

#merges?:
# anger: annoyance, disgust, disapproval
# joy: relief, gratitude, optimism
# love: caring
# sadness: disappointment, grief, remorse
# confusion: surprise
# quite a few emotions can be


# Model training parameters.
CONFIG = {
    'name': 'models.prado',
    'batch_size': 1024,
    'train_steps': 10000,
    'learning_rate': 0.0006,
    'learning_rate_decay_steps': 340,
    'learning_rate_decay_rate': 0.7,
}

# Limits the amount of logging output produced by the training run, in order to
# avoid browser slowdowns.
CONFIG['save_checkpoints_steps'] = int(CONFIG['train_steps'] / 10)

MODEL_CONFIG = {
    'labels': LABELS,
    'multilabel': True,
    'quantize': False,
    'max_seq_len': 128,
    'max_seq_len_inference': 128,
    'exclude_nonalphaspace_unicodes': False,
    'split_on_space': True,
    'embedding_regularizer_scale': 0.035,
    'embedding_size': 64,
    'bigram_channels': 64,
    'trigram_channels': 64,
    'feature_size': 512,
    'network_regularizer_scale': 0.0001,
    'keep_prob': 0.5,
    'word_novelty_bits': 0,
    'doc_size_levels': 0,
    'add_bos_tag': False,
    'add_eos_tag': False,
    'pre_logits_fc_layers': [],
    'text_distortion_probability': 0.0,
}

CONFIG['model_config'] = MODEL_CONFIG

In [ ]:
model = build_model(base_layers.PREDICT)
model.load_weights('model_checkpoint')

In [ ]:
import numpy as np

PREDICT_TEXT = [
  b'depressed in my room',
  b'Happy birthday!',
  b'Go gators!',
]

for text in PREDICT_TEXT:
  results = model.predict(x=[text])
  print('')
  print('{}:'.format(text))
  labels = np.flip(np.argsort(results[0]))
  for x in range(3):
    label = LABELS[labels[x]]
    #label = EMOJI_MAP[label] if EMOJI_MAP[label] else label
    print('{}: {}'.format(label, results[0][labels[x]]))

1/1 [==============================] - 0s 46ms/step

b'depressed in my room':
neutral: 0.7431514859199524
sadness: 0.4069349467754364
disappointment: 0.366469144821167
1/1 [==============================] - 0s 50ms/step

b'Happy birthday!':
joy: 0.9721881747245789
excitement: 0.7735481858253479
admiration: 0.4375458061695099
1/1 [==============================] - 0s 41ms/step

b'Go gators!':
neutral: 0.865899384021759
disapproval: 0.4562932848930359
caring: 0.4002096652984619
